In [72]:
import os
import requests
import re
import csv
import json
from geojson import Feature, FeatureCollection, Point

In [2]:
skiareas = []
with open("rawskidata.txt") as f:
    for line in f:
        match = re.search('<li>', line)
        if(match):
            match = re.search('href\S+', line)
            if(match):
                skiareas.append(match.group(0)[6:-1])

In [80]:
skiareas[0][6:]

'Mohawk_Mountain_Ski_Area'

In [60]:
for num in range(len(skiareas)):
    URL = 'https://en.wikipedia.org' + skiareas[num]
    data = requests.get(URL, stream=True)
    #     college_sb_data.append(pd.Series(name=str(yr)))
    for line in data.iter_lines():
        match = re.search('Vertical</th><td>(.[\d,]*)', str(line))
        if(match):
            if(len(match.group(1)) > 1):
                vert_value = match.group(1).replace(',', '')
                print(vert_value)

650
425
550
625
1380
845
240
2000
1300
2820
465
309
750
1000
1150


KeyboardInterrupt: 

In [93]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
for num in range(len(skiareas)):
    viewcount = 0
    URL = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/' + skiareas[num][6:] + '/monthly/2020010100/2020123100'
    data = requests.get(url = URL, headers = headers)
    data_json = data.json()
    #     college_sb_data.append(pd.Series(name=str(yr)))
    for month in data_json['items']:
        viewcount += month['views']
    print(viewcount)
    break

1009
1948
2386
2724
3250
3565
3962
4438
4871
5358
5887
7258
7258


In [94]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
ski_areas_formatted = []
for num in range(len(skiareas)):
    URL = 'https://en.wikipedia.org' + skiareas[num]
    data = requests.get(URL, stream=True)
    name = None
    coords = None
    vert = None
    viewcount = 0
    for line in data.iter_lines():
        
        if line:
            line = str(line)[2:]
            match = re.search('<title>', line)
            if(match):
                name = line[7:-21]
            match = re.search('for this location">\d.+', line)
            if(match):
                coords = (match.group(0)[19:45]).split('\\xc2\\xb0N ')
            match = re.search('Vertical</th><td>(.[^<]*)', str(line))
            match = re.search('Vertical</th><td>(.[\d,]*)', str(line))
            if(match):
                if(len(match.group(1)) > 1):
                    vert = match.group(1).replace(',', '')
    
    URL = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/' + skiareas[num][6:] + '/monthly/2020010100/2020123100'
    data = requests.get(url = URL, headers = headers)
    if data.status_code == 200:
        data_json = data.json()
        for month in data_json['items']:
            viewcount += month['views']
        
    if name and coords and vert:
        ski_areas_formatted.append({'name': name, 'lat': coords[0], 'lng': coords[1], 'pageviews': viewcount, 'vert': vert})
        name = None
        coords = None
        vert = None
        viewcount = 0
    elif name and coords:
        ski_areas_formatted.append({'name': name, 'lat': coords[0], 'lng': coords[1], 'pageviews': viewcount})
        name = None
        coords = None
        viewcount = 0
    
            
    print('on', num, 'out of,', len(skiareas))
    print(ski_areas_formatted[len(ski_areas_formatted)-1])

on 0 out of, 453
{'name': 'Mohawk Mountain Ski Area', 'lat': '41.83675', 'lng': '73.31342', 'pageviews': 7258, 'vert': '650'}
on 1 out of, 453
{'name': 'Mount Southington', 'lat': '41.58235', 'lng': '72.92587', 'pageviews': 2923, 'vert': '425'}
on 2 out of, 453
{'name': 'Powder Ridge Ski Area', 'lat': '41.502006', 'lng': '72.7371', 'pageviews': 4329, 'vert': '550'}
on 3 out of, 453
{'name': 'Ski Sundown', 'lat': '41.88407', 'lng': '72.94523', 'pageviews': 4156, 'vert': '625'}
on 4 out of, 453
{'name': 'Baker Mountain (ski area)', 'lat': '45.08472', 'lng': '69.90556', 'pageviews': 1705}
on 5 out of, 453
{'name': 'Big Rock (ski resort)', 'lat': '46.52222', 'lng': '67.82778', 'pageviews': 2536}
on 6 out of, 453
{'name': 'Big Squaw Mountain Resort', 'lat': '45.50417', 'lng': '69.70556', 'pageviews': 347}
on 7 out of, 453
{'name': 'Black Mountain of Maine', 'lat': '44.58000', 'lng': '70.61750', 'pageviews': 5292, 'vert': '1380'}
on 8 out of, 453
{'name': 'Camden Snow Bowl', 'lat': '44.21721

on 70 out of, 453
{'name': 'Okemo Mountain', 'lat': '43.40139', 'lng': '72.71667', 'pageviews': 22238, 'vert': '2200'}
on 71 out of, 453
{'name': 'Pico Mountain', 'lat': '43.66167', 'lng': '72.84250', 'pageviews': 5638, 'vert': '1967'}
on 72 out of, 453
{'name': 'Plymouth Notch, Vermont', 'lat': '43.5299', 'lng': '72.7388\\x', 'pageviews': 1680}
on 73 out of, 453
{'name': 'Quechee Ski Area', 'lat': '43.6572', 'lng': '72.4431\\x', 'pageviews': 588}
on 74 out of, 453
{'name': "Smugglers' Notch Resort", 'lat': '44.57278', 'lng': '72.77611', 'pageviews': 5197, 'vert': '2610'}
on 75 out of, 453
{'name': 'Stowe Mountain Resort', 'lat': '44.53194', 'lng': '72.78750', 'pageviews': 22890, 'vert': '2360'}
on 76 out of, 453
{'name': 'Stratton Mountain Resort', 'lat': '43.11417', 'lng': '72.90667', 'pageviews': 13950, 'vert': '2003'}
on 77 out of, 453
{'name': 'Sugarbush Resort', 'lat': '44.13722', 'lng': '72.90667', 'pageviews': 20574, 'vert': '2600'}
on 78 out of, 453
{'name': 'Suicide Six', 'la

on 138 out of, 453
{'name': 'Montage Mountain Ski Resort', 'lat': '41.3533', 'lng': '75.6592\\x', 'pageviews': 9226, 'vert': '1000'}
on 139 out of, 453
{'name': 'Tussey Mountain Ski Area', 'lat': '40.7682', 'lng': '77.7520\\x', 'pageviews': 2047, 'vert': '500'}
on 140 out of, 453
{'name': 'Whitetail Ski Resort', 'lat': '39.74167', 'lng': '77.93278', 'pageviews': 19522, 'vert': '935'}
on 141 out of, 453
{'name': 'Whitetail Ski Resort', 'lat': '39.74167', 'lng': '77.93278', 'pageviews': 19522, 'vert': '935'}
on 142 out of, 453
{'name': 'Appalachian Ski Mountain', 'lat': '36.174', 'lng': '81.662\\xc2', 'pageviews': 5140}
on 143 out of, 453
{'name': 'Cataloochee Ski Area', 'lat': '35.563', 'lng': '83.090\\xc2', 'pageviews': 4812}
on 144 out of, 453
{'name': 'Cataloochee Ski Area', 'lat': '35.563', 'lng': '83.090\\xc2', 'pageviews': 4812}
on 145 out of, 453
{'name': 'Cataloochee Ski Area', 'lat': '35.563', 'lng': '83.090\\xc2', 'pageviews': 4812}
on 146 out of, 453
{'name': 'Cataloochee Ski

on 209 out of, 453
{'name': 'Afton Alps', 'lat': '44.85556', 'lng': '92.78917', 'pageviews': 7822, 'vert': '350'}
on 210 out of, 453
{'name': 'Buck Hill', 'lat': '44.724', 'lng': '93.283\\xc2', 'pageviews': 8710}
on 211 out of, 453
{'name': 'Buena Vista Ski Area', 'lat': '47.65806', 'lng': '94.87917', 'pageviews': 797}
on 212 out of, 453
{'name': 'Buena Vista Ski Area', 'lat': '47.65806', 'lng': '94.87917', 'pageviews': 797}
on 213 out of, 453
{'name': 'Buena Vista Ski Area', 'lat': '47.65806', 'lng': '94.87917', 'pageviews': 797}
on 214 out of, 453
{'name': 'Buena Vista Ski Area', 'lat': '47.65806', 'lng': '94.87917', 'pageviews': 797}
on 215 out of, 453
{'name': 'Buena Vista Ski Area', 'lat': '47.65806', 'lng': '94.87917', 'pageviews': 797}
on 216 out of, 453
{'name': 'Hyland Hills Ski Area', 'lat': '44.844', 'lng': '93.364\\xc2', 'pageviews': 615}
on 217 out of, 453
{'name': 'Lutsen Mountains', 'lat': '47.664', 'lng': '90.714\\xc2', 'pageviews': 13685, 'vert': '825'}
on 218 out of, 

on 282 out of, 453
{'name': 'Buttermilk (ski area)', 'lat': '39.20500', 'lng': '106.8605', 'pageviews': 11400, 'vert': '2030'}
on 283 out of, 453
{'name': 'Snowmass (ski area)', 'lat': '39.1956', 'lng': '106.9513\\', 'pageviews': 18639, 'vert': '4406'}
on 284 out of, 453
{'name': 'Beaver Creek Resort', 'lat': '39.63139', 'lng': '106.5216', 'pageviews': 25201, 'vert': '3340'}
on 285 out of, 453
{'name': 'Breckenridge Ski Resort', 'lat': '39.480', 'lng': '106.067\\xc', 'pageviews': 37999}
on 286 out of, 453
{'name': 'Breckenridge Ski Resort', 'lat': '39.480', 'lng': '106.067\\xc', 'pageviews': 37999}
on 287 out of, 453
{'name': 'Copper Mountain (Colorado)', 'lat': '39.50167', 'lng': '106.1563', 'pageviews': 36168}
on 288 out of, 453
{'name': 'Copper Mountain (Colorado)', 'lat': '39.50167', 'lng': '106.1563', 'pageviews': 36168}
on 289 out of, 453
{'name': 'Crested Butte Mountain Resort', 'lat': '38.89972', 'lng': '106.9650', 'pageviews': 15200}
on 290 out of, 453
{'name': 'Echo Mountain 

on 353 out of, 453
{'name': 'Ski Cloudcroft', 'lat': '32.9507083', 'lng': '105.70', 'pageviews': 0}
on 354 out of, 453
{'name': 'Ski Santa Fe', 'lat': '35.7963250', 'lng': '105.80', 'pageviews': 0}
on 355 out of, 453
{'name': 'Taos Ski Valley, New Mexico', 'lat': '36.59778', 'lng': '105.4450', 'pageviews': 37879}
on 356 out of, 453
{'name': 'Alta Ski Area', 'lat': '40.58083', 'lng': '111.6372', 'pageviews': 35570, 'vert': '2538'}
on 357 out of, 453
{'name': 'Beaver Mountain', 'lat': '41.968', 'lng': '111.541\\xc', 'pageviews': 11413, 'vert': '1600'}
on 358 out of, 453
{'name': 'Brian Head Ski Resort', 'lat': '37.683', 'lng': '112.833\\xc', 'pageviews': 10412, 'vert': '1707'}
on 359 out of, 453
{'name': 'Brighton Ski Resort', 'lat': '40.60000', 'lng': '111.5833', 'pageviews': 18008, 'vert': '1750'}
on 360 out of, 453
{'name': 'Cherry Peak Resort', 'lat': '41.92639', 'lng': '111.7566', 'pageviews': 7334, 'vert': '1265'}
on 361 out of, 453
{'name': 'Deer Valley', 'lat': '40.62278', 'lng':

on 422 out of, 453
{'name': 'Anthony Lakes (ski area)', 'lat': '44.9637', 'lng': '118.2348\\', 'pageviews': 3770, 'vert': '900'}
on 423 out of, 453
{'name': 'Cooper Spur ski area', 'lat': '45.41228', 'lng': '121.6050', 'pageviews': 3308}
on 424 out of, 453
{'name': 'Ferguson Ridge Ski Area', 'lat': '45.28209', 'lng': '117.1128', 'pageviews': 1708}
on 425 out of, 453
{'name': 'Hoodoo (ski area)', 'lat': '44.409', 'lng': '121.872\\xc', 'pageviews': 6586, 'vert': '1035'}
on 426 out of, 453
{'name': 'Mount Ashland Ski Area', 'lat': '42.08333', 'lng': '122.7166', 'pageviews': 4683}
on 427 out of, 453
{'name': 'Mount Bachelor ski area', 'lat': '44.003', 'lng': '121.677\\xc', 'pageviews': 17885, 'vert': '3365'}
on 428 out of, 453
{'name': 'Mount Hood Meadows', 'lat': '45.32889', 'lng': '121.6625', 'pageviews': 17376}
on 429 out of, 453
{'name': 'Mount Hood Skibowl', 'lat': '45.30189', 'lng': '121.7732', 'pageviews': 14370}
on 430 out of, 453
{'name': 'Spout Springs Ski Area', 'lat': '45.7550'

In [95]:
for area in ski_areas_formatted:
    while (not str.isdigit(area['lng'][len(area['lng'])-1])) or area['lng'][-2:] == 'c2':
        area['lng'] = area['lng'][:-1]
    area['lng'] = -float(area['lng'])


In [96]:
skiarea_csv = []
num = 0
for area in ski_areas_formatted:
    if 'vert' in area:
        skiarea_csv.append([area['name'], area['lat'], area['lng'], area['pageviews'], area['vert']])
    else:
        skiarea_csv.append([area['name'], area['lat'], area['lng'], area['pageviews']])
    print(skiarea_csv[num])
    num += 1

['Mohawk Mountain Ski Area', '41.83675', -73.31342, 7258, '650']
['Mount Southington', '41.58235', -72.92587, 2923, '425']
['Powder Ridge Ski Area', '41.502006', -72.7371, 4329, '550']
['Ski Sundown', '41.88407', -72.94523, 4156, '625']
['Baker Mountain (ski area)', '45.08472', -69.90556, 1705]
['Big Rock (ski resort)', '46.52222', -67.82778, 2536]
['Big Squaw Mountain Resort', '45.50417', -69.70556, 347]
['Black Mountain of Maine', '44.58000', -70.6175, 5292, '1380']
['Camden Snow Bowl', '44.217216', -69.1346, 2614, '845']
['Eaton Mountain', '44.76806', -69.61944, 1354]
['Lonesome Pine Trails', '47.23278', -68.59306, 1173]
['Lost Valley Ski Area', '44.136', -70.281, 2093, '240']
['Ski Mount Abram', '44.37917', -70.70694, 239]
['Powderhouse Hill (ski area)', '43.23694', -70.80222, 1442]
['Saddleback Maine (ski resort)', '44.93667', -70.50306, 11261, '2000']
['Shawnee Peak Ski Area', '44.05833', -70.81528, 5404, '1300']
['Sugarloaf (ski resort)', '45.03139', -70.31306, 31438, '2820']
['

In [70]:
fields = ['name', 'lat', 'lng', 'vert']
with open('skiareas_sorted_vert.csv', 'w') as f: 
      
    # using csv.writer method from CSV package 
    write = csv.writer(f) 
      
    write.writerow(fields) 
    write.writerows(skiarea_csv) 

In [19]:
cityjsonsorted = []
citynum = 0
for city in cityjson:
    cityjsonsorted.append([])
    cityjsonsorted[citynum].append(city['city'])
    cityjsonsorted[citynum].append(city['growth_from_2000_to_2013'])
    cityjsonsorted[citynum].append(city['latitude'])
    cityjsonsorted[citynum].append(city['longitude'])
    cityjsonsorted[citynum].append(city['population'])
    cityjsonsorted[citynum].append(city['rank'])
    cityjsonsorted[citynum].append(city['state'])
    citynum += 1

In [25]:
new_fields = ['city_name', 'growth_2000_2013', 'lat', 'lng', 'pop', 'rank_size', 'state']
with open('cities_sorted.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(new_fields)
    write.writerows(cityjsonsorted)

In [97]:
features = []
for area in ski_areas_formatted:
    latitude, longitude = map(float, (area['lat'], area['lng']))
    if 'vert' in area:
        features.append(
            Feature(
                geometry = Point((longitude, latitude)),
                properties = {
                    'name': area['name'],
                    'pageviews-2020': area['pageviews'],
                    'vertical': area['vert']
                }
            )
        )
    else:
        features.append(
            Feature(
                geometry = Point((longitude, latitude)),
                properties = {
                    'name': area['name'],
                    'pageviews-2020': area['pageviews']
                }
            )
        )

collection = FeatureCollection(features)
with open("ski_geojson.json", "w") as f:
    f.write('%s' % collection)